# MORPC Insights - Race and Ethnicity

## Overview

The U.S. Census Population Estimates Program (PEP) produces a data product called [County Population by Characteristics](https://www.census.gov/data/tables/time-series/demo/popest/2020s-counties-detail.html) which contains county level estimates of population by age, race, and sex.  Each year the program releases a new "vintage" of the estimates which includes the most recent year and revised estimates for prior years going back to the last decennial census.
  
This notebook produces a tileset that includes a summary of population by race and ethnicity for the MORPC 15-county region and the counties therein.  This notebook is the final stage in a pipeline that fetches, standardizes, and summarizes the PEP population characteristics data.

## Setup

### Load required libraries

In [1]:
import pandas as pd
import numpy as np
import frictionless
import os
import sys
import json
import datetime
import textwrap
import matplotlib
from matplotlib import pyplot as plt
sys.path.append(os.path.normpath("../morpc-common"))
import morpc

### User-specified parameters

In [2]:
YEAR_RANGE = [2010, 2023]

### Static parameters

In [3]:
INPUT_DIR = os.path.normpath("./input_data")

OUTPUT_DIR = os.path.normpath("./output_data")

CHART_DIRNAME = "charts"

# The following map will be used to combine various race categories into a simplified set.
# The new set will consist of White, Asian, Black or African American alone (all non-Hispanic),
# plus some other race alone (non-Hispanic), plus two or more races (non-Hispanic), plus Hispanic
# or Latino (any races).
RACE_MAP = {
    "White": ["NHWA"],
    "Asian": ["NHAA"],
    "Black or African American": ["NHBA"],
    "Some other race": ["NHIA","NHNA"],
    "Two or more races": ["NHTOM"],
    "Hispanic or Latino (any races)": ["HAA","HBA","HIA","HNA","HTOM","HWA"]
}

### Define inputs

#### Create input data directory

Create input data directory if it doesn't exist.

In [4]:
inputDir = os.path.normpath(INPUT_DIR)
if not os.path.exists(inputDir):
    os.makedirs(inputDir)

#### Census PEP population characteristics data (standardized)

In [5]:
PEP_INPUT_RESOURCE = os.path.normpath("../morpc-censuspep-standardize/output_data/censusPep_county_char.resource.yaml")
print("Resource file: {}".format(PEP_INPUT_RESOURCE))

Resource file: ..\morpc-censuspep-standardize\output_data\censusPep_county_char.resource.yaml


### Define outputs

#### Create output data directory

Create output data directory if it doesn't exist.

In [7]:
outputDir = os.path.normpath(OUTPUT_DIR)
if not os.path.exists(outputDir):
    os.makedirs(outputDir)   

In [8]:
chartDir = os.path.join(outputDir, CHART_DIRNAME)
if not os.path.exists(chartDir):
    os.makedirs(chartDir)    

#### Population characteristics data (summarized)

In [9]:
OUTPUT_TABLE_FILENAME = "insights-race.csv"
OUTPUT_TABLE_PATH = os.path.join(outputDir, OUTPUT_TABLE_FILENAME)
OUTPUT_TABLE_SCHEMA_PATH = OUTPUT_TABLE_PATH.replace(".csv",".schema.yaml")
OUTPUT_TABLE_RESOURCE_PATH = OUTPUT_TABLE_PATH.replace(".csv",".resource.yaml")
print("Data: {}".format(OUTPUT_TABLE_PATH))
print("Schema: {}".format(OUTPUT_TABLE_SCHEMA_PATH))
print("Resource file: {}".format(OUTPUT_TABLE_RESOURCE_PATH))

Data: output_data\insights-race.csv
Schema: output_data\insights-race.schema.yaml
Resource file: output_data\insights-race.resource.yaml


## Prepare input data

### Load standardized PEP data from upstream workflows

In [13]:
(pepRaw, pepRawResource, pepRawSchema) = morpc.frictionless_load_data(PEP_INPUT_RESOURCE, validate=True, archiveDir=inputDir)

morpc.load_frictionless_data | INFO | Loading Frictionless Resource file at location ..\morpc-censuspep-standardize\output_data\censusPep_county_char.resource.yaml
morpc.load_frictionless_data | INFO | Copying data, resource file, and schema to directory input_data
morpc.load_frictionless_data | INFO | --> Data file: input_data\censusPep_county_char.csv
morpc.load_frictionless_data | INFO | --> Resource file: input_data\censusPep_county_char.resource.yaml
morpc.load_frictionless_data | INFO | --> Schema file: input_data\censusPep_county_char.schema.yaml
morpc.load_frictionless_data | INFO | Validating resource including data and schema.
morpc.frictionless_validate_resource | INFO | Validating resource on disk (including data and schema). This may take some time.
morpc.frictionless_validate_resource | INFO | Resource is valid
morpc.load_frictionless_data | INFO | Loading data.
frictionless_cast_field_types | INFO | Casting field GEOID as type string.
frictionless_cast_field_types | INFO

In [340]:
pepRaw.head()

,GEOID,STATE,COUNTY,SEX,RACE,AGEGROUP,SOURCE,VINTAGE,YEAR,POP
0,39001,Ohio,Adams,F,AA,0 to 4,INT,2012,2000,1
1,39001,Ohio,Adams,F,AA,0 to 4,INT,2012,2001,1
2,39001,Ohio,Adams,F,AA,0 to 4,INT,2012,2002,1
3,39001,Ohio,Adams,F,AA,0 to 4,INT,2012,2003,1
4,39001,Ohio,Adams,F,AA,0 to 4,INT,2012,2004,1


In [494]:
pep = pepRaw.copy()

## Transform data to format required by Insights platform

### Extract records of interest

Extract records for MORPC counties.

In [495]:
pep = pep.loc[pep["COUNTY"].isin(morpc.CONST_REGIONS["15-County Region"])].copy()

Extract records for the specified range of years.

In [496]:
pep = pep.loc[pep["YEAR"].isin(range(YEAR_RANGE[0], YEAR_RANGE[1]+1))].copy()

The remaining records include the following vintages for each reference year.

In [497]:
for year in pep["YEAR"].unique():
    vintages = list(pep.loc[pep["YEAR"] == year, "VINTAGE"].unique())
    print("{}: {}".format(year, vintages))

2010: [np.int64(2012), np.int64(2020)]
2011: [np.int64(2020)]
2012: [np.int64(2020)]
2013: [np.int64(2020)]
2014: [np.int64(2020)]
2015: [np.int64(2020)]
2016: [np.int64(2020)]
2017: [np.int64(2020)]
2018: [np.int64(2020)]
2019: [np.int64(2020)]
2020: [np.int64(2020), np.int64(2023)]
2021: [np.int64(2023)]
2022: [np.int64(2023)]
2023: [np.int64(2023)]


Extract the estimate for each year from the latest available vintage.

In [498]:
pep = morpc.extract_vintage(pep, vintagePeriodField="VINTAGE", refPeriodField="YEAR")

The remaining records should include only a single vintage for each reference year.

In [499]:
for year in pep["YEAR"].unique():
    vintages = list(pep.loc[pep["YEAR"] == year, "VINTAGE"].unique())
    print("{}: {}".format(year, vintages))

2010: [np.int64(2020)]
2011: [np.int64(2020)]
2012: [np.int64(2020)]
2013: [np.int64(2020)]
2014: [np.int64(2020)]
2015: [np.int64(2020)]
2016: [np.int64(2020)]
2017: [np.int64(2020)]
2018: [np.int64(2020)]
2019: [np.int64(2020)]
2020: [np.int64(2023)]
2021: [np.int64(2023)]
2022: [np.int64(2023)]
2023: [np.int64(2023)]


The data includes totals by sex but no totals by age group. Drop the totals.  We'll recompute the ones we need.

In [500]:
for var in ["SEX","RACE","AGEGROUP"]:
    print("{}: {}".format(var, list(pep[var].unique())))

SEX: ['F', 'M', 'TOT']
RACE: ['AA', 'BA', 'HAA', 'HBA', 'HIA', 'HNA', 'HTOM', 'HWA', 'IA', <NA>, 'NHAA', 'NHBA', 'NHIA', 'NHNA', 'NHTOM', 'NHWA', 'TOM', 'TOT', 'WA']
AGEGROUP: ['0 to 4', '10 to 14', '15 to 19', '20 to 24', '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49', '5 to 9', '50 to 54', '55 to 59', '60 to 64', '65 to 69', '70 to 74', '75 to 79', '80 to 84', '85+']


Extract the totals records for sex.

In [501]:
pep = pep.loc[(pep["SEX"] != "TOT") & (pep["RACE"] != "TOT")]

Verify that the totals have been removed.

In [502]:
for var in ["SEX","RACE","AGEGROUP"]:
    print("{}: {}".format(var, list(pep[var].unique())))

SEX: ['F', 'M']
RACE: ['AA', 'BA', 'HAA', 'HBA', 'HIA', 'HNA', 'HTOM', 'HWA', 'IA', 'NHAA', 'NHBA', 'NHIA', 'NHNA', 'NHTOM', 'NHWA', 'TOM', 'WA']
AGEGROUP: ['0 to 4', '10 to 14', '15 to 19', '20 to 24', '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49', '5 to 9', '50 to 54', '55 to 59', '60 to 64', '65 to 69', '70 to 74', '75 to 79', '80 to 84', '85+']


Compute totals by race.

In [503]:
pep = pep.drop(columns=["SEX","AGEGROUP"]).groupby(["GEOID","COUNTY","YEAR","VINTAGE","SOURCE","RACE"]).sum().reset_index()

Add the fully-qualified GEOIDs and SUMLEVELS for the geographies.

In [504]:
pep["SUMLEVEL"] = morpc.SUMLEVEL_LOOKUP["COUNTY"]
pep["GEOIDFQ"] = pep["SUMLEVEL"] + "0000US" + pep["GEOID"]

Retain only the fields that we require.

In [505]:
pep = pep.filter(items=["GEOIDFQ","SUMLEVEL","COUNTY","YEAR","RACE","POP"], axis="columns")

Inspect the data.

In [506]:
pep.head()

,GEOIDFQ,SUMLEVEL,COUNTY,YEAR,RACE,POP
0,0500000US39041,050,Delaware,2010,AA,7741
1,0500000US39041,050,Delaware,2010,BA,5964
2,0500000US39041,050,Delaware,2010,HAA,59
3,0500000US39041,050,Delaware,2010,HBA,101
4,0500000US39041,050,Delaware,2010,HIA,89


### Compute totals for 15-County Region

Sum the population for each sex/race/age group combination for all of the counties to obtain the totals for the 15-county region.

In [507]:
temp = pep.groupby(['RACE','YEAR'])['POP'].sum().reset_index()

Add identifier values for the 15-county summary data.

In [508]:
temp["COUNTY"] = '15-County Region'
temp["GEOIDFQ"] = 'M010000US001'
temp["SUMLEVEL"] = 'M01'

Append the 15-county summary data to the county data.

In [509]:
pep = pd.concat([pep, temp], axis="index")

Inspect the data.

In [510]:
pep

,GEOIDFQ,SUMLEVEL,COUNTY,YEAR,RACE,POP
0,0500000US39041,050,Delaware,2010,AA,7741
1,0500000US39041,050,Delaware,2010,BA,5964
2,0500000US39041,050,Delaware,2010,HAA,59
3,0500000US39041,050,Delaware,2010,HBA,101
4,0500000US39041,050,Delaware,2010,HIA,89
...,...,...,...,...,...,...
233,M010000US001,M01,15-County Region,2019,WA,1857577
234,M010000US001,M01,15-County Region,2020,WA,1864776
235,M010000US001,M01,15-County Region,2021,WA,1857247
236,M010000US001,M01,15-County Region,2022,WA,1856850


### Pivot to semi-wide format

In [511]:
pep = pep.pivot(index=["GEOIDFQ","SUMLEVEL","COUNTY","YEAR"], columns="RACE", values="POP")
pep.head()

RACE                                      AA    BA  HAA  HBA  HIA  HNA  HTOM  \
GEOIDFQ        SUMLEVEL COUNTY   YEAR                                          
0500000US39041 050      Delaware 2010   7741  5964   59  101   89   11   156   
                                 2011   8299  6126   66  125   93   14   177   
                                 2012   8695  6186   74  138  108   14   174   
                                 2013   9463  6348   76  162  110   19   193   
                                 2014  10348  6473   80  176   99   16   200   

RACE                                    HWA   IA   NHAA  NHBA  NHIA  NHNA  \
GEOIDFQ        SUMLEVEL COUNTY   YEAR                                       
0500000US39041 050      Delaware 2010  3289  309   7682  5863   220    50   
                                 2011  3417  326   8233  6001   233    58   
                                 2012  3623  354   8621  6048   246    58   
                                 2013  3770  381   9387  6186   271    64   
                                 2014  3960  380  10268  6297   281    65   

RACE                                   NHTOM    NHWA   TOM      WA  
GEOIDFQ        SUMLEVEL COUNTY   YEAR                               
0500000US39041 050      Delaware 2010   2760  154834  2916  158123  
                                 2011   2896  157271  3073  160688  
                                 2012   3028  159102  3202  162725  
                                 2013   3199  162108  3392  165878  
                                 2014   3354  164853  3554  168813

### Combine the race categories

The data includes many race categories. This is too many to plot, so we'll combine some.

In [512]:
temp = pep.copy()
for combo,races in RACE_MAP.items():
    print("Creating category '{}' by combining races {}".format(combo, ",".join(races)))
    pep[combo] = temp[races].sum(axis="columns").astype("int")

Creating category 'White' by combining races NHWA
Creating category 'Asian' by combining races NHAA
Creating category 'Black or African American' by combining races NHBA
Creating category 'Some other race' by combining races NHIA,NHNA
Creating category 'Two or more races' by combining races NHTOM
Creating category 'Hispanic or Latino (any races)' by combining races HAA,HBA,HIA,HNA,HTOM,HWA


Extract only the combined race columns and reset the index.

In [513]:
pep = pep.filter(items=RACE_MAP.keys(), axis="columns").reset_index()

Inspect the data.

In [514]:
pep.head()

,GEOIDFQ,SUMLEVEL,COUNTY,YEAR,White,Asian,Black or African American,Some other race,Two or more races,Hispanic or Latino (any races)
0,0500000US39041,050,Delaware,2010,154834,7682,5863,270,2760,3705
1,0500000US39041,050,Delaware,2011,157271,8233,6001,291,2896,3892
2,0500000US39041,050,Delaware,2012,159102,8621,6048,304,3028,4131
3,0500000US39041,050,Delaware,2013,162108,9387,6186,335,3199,4330
4,0500000US39041,050,Delaware,2014,164853,10268,6297,346,3354,4531


### Compute proportions

In [515]:
for race in RACE_MAP.keys():
    print("Computing proportion for race {}".format(race))
    pep["{} proportion".format(race)] = pep[race] / pep[RACE_MAP.keys()].sum(axis="columns")

Computing proportion for race White
Computing proportion for race Asian
Computing proportion for race Black or African American
Computing proportion for race Some other race
Computing proportion for race Two or more races
Computing proportion for race Hispanic or Latino (any races)


In [516]:
pep.head()

,GEOIDFQ,SUMLEVEL,COUNTY,YEAR,White,Asian,Black or African American,Some other race,Two or more races,Hispanic or Latino (any races),White proportion,Asian proportion,Black or African American proportion,Some other race proportion,Two or more races proportion,Hispanic or Latino (any races) proportion
0,0500000US39041,050,Delaware,2010,154834,7682,5863,270,2760,3705,0.884190,0.043869,0.033481,0.001542,0.015761,0.021158
1,0500000US39041,050,Delaware,2011,157271,8233,6001,291,2896,3892,0.880656,0.046102,0.033603,0.001629,0.016216,0.021794
2,0500000US39041,050,Delaware,2012,159102,8621,6048,304,3028,4131,0.877882,0.047568,0.033371,0.001677,0.016708,0.022794
3,0500000US39041,050,Delaware,2013,162108,9387,6186,335,3199,4330,0.873686,0.050592,0.033340,0.001805,0.017241,0.023337
4,0500000US39041,050,Delaware,2014,164853,10268,6297,346,3354,4531,0.869253,0.054142,0.033203,0.001824,0.017685,0.023892


Make sure the division didn't produce any divide by zero errors.

In [517]:
for race in RACE_MAP.keys():
    print("{}: null count={}, inf count={}".format(
        race, 
        pep.loc[pep[race].isna()].shape[0], 
        pep.loc[pep[race].isin([np.nan, -np.nan])].shape[0]
    ))

White: null count=0, inf count=0
Asian: null count=0, inf count=0
Black or African American: null count=0, inf count=0
Some other race: null count=0, inf count=0
Two or more races: null count=0, inf count=0
Hispanic or Latino (any races): null count=0, inf count=0


### Melt back to long format

Melt back to long format, dropping the counts and retaining the proportions.

In [518]:
pep = pep.filter(regex="GEOIDFQ|SUMLEVEL|COUNTY|YEAR|proportion$")
pep = pep.rename(columns=(lambda x:x.replace(" proportion","")))
pep = pep.melt(id_vars=["GEOIDFQ","SUMLEVEL","COUNTY","YEAR"], var_name="RACE", value_name="PROPORTION")
pep

,GEOIDFQ,SUMLEVEL,COUNTY,YEAR,RACE,PROPORTION
0,0500000US39041,050,Delaware,2010,White,0.884190
1,0500000US39041,050,Delaware,2011,White,0.880656
2,0500000US39041,050,Delaware,2012,White,0.877882
3,0500000US39041,050,Delaware,2013,White,0.873686
4,0500000US39041,050,Delaware,2014,White,0.869253
...,...,...,...,...,...,...
1339,M010000US001,M01,15-County Region,2019,Hispanic or Latino (any races),0.041684
1340,M010000US001,M01,15-County Region,2020,Hispanic or Latino (any races),0.048386
1341,M010000US001,M01,15-County Region,2021,Hispanic or Latino (any races),0.049611
1342,M010000US001,M01,15-County Region,2022,Hispanic or Latino (any races),0.051234


### Standardize the data

Load the output schema.

In [519]:
outputSchema = morpc.frictionless_load_schema(OUTPUT_TABLE_SCHEMA_PATH)

Give the "COUNTY" column a more generic name.

In [520]:
pep = pep.rename(columns={"COUNTY":"GEONAME"})

Convert SUMLEVEL to geography type.

In [521]:
pep["GEOTYPE"] = pep["SUMLEVEL"].apply(lambda x:("Region" if x==morpc.SUMLEVEL_LOOKUP["REGION15"] else "County"))

Append a "County" suffix to county names.

In [522]:
pep["GEONAME"] = pep["GEONAME"].apply(lambda x:(x if x == "15-County Region" else "{} County".format(x)))

Make sure the data contains only the columns defined in the schema.

In [523]:
pep = pep.filter(items=outputSchema.field_names, axis="columns")

Make sure the data types comply with the schema.

In [524]:
pep = morpc.cast_field_types(pep, outputSchema)

frictionless_cast_field_types | INFO | Casting field GEOIDFQ as type string.
frictionless_cast_field_types | INFO | Casting field GEOTYPE as type string.
frictionless_cast_field_types | INFO | Casting field GEONAME as type string.
frictionless_cast_field_types | INFO | Casting field YEAR as type integer.
frictionless_cast_field_types | INFO | Casting field RACE as type string.
frictionless_cast_field_types | INFO | Casting field PROPORTION as type number.


## Export data

In [525]:
pep.to_csv(OUTPUT_TABLE_PATH, index=False)

## Create resource file for exported data

In [526]:
pepResource = morpc.frictionless_create_resource(OUTPUT_TABLE_FILENAME, 
    resourcePath=OUTPUT_TABLE_RESOURCE_PATH,
    title="MORPC Insights | Race and Ethnicity", 
    name="race", 
    description="Count and proportions of racial subgroups of the MORPC 15-County region and the counties therein, according to the U.S. Census Population Estimates Program.",
    writeResource=True,
    validate=True
)

morpc.frictionless_create_resource | INFO | Format not specified. Using format derived from data file extension: csv
morpc.frictionless_create_resource | INFO | Schema path not specified. Using path derived from data file path: insights-race.schema.yaml
morpc.frictionless_create_resource | INFO | Writing Frictionless Resource file to output_data\insights-race.resource.yaml
morpc.frictionless_create_resource | INFO | Validating resource on disk.
morpc.frictionless_validate_resource | INFO | Validating resource on disk (including data and schema). This may take some time.
morpc.frictionless_validate_resource | INFO | Resource is valid


## Generate static charts

In [527]:
for f in os.scandir(chartDir):
    os.remove(f)

Load a standard color set for the chart elements.

In [528]:
colorset = ["#149ece","#007ac2","#a7c636","#9e559c","#fc921f","#ffde3e"]

In [529]:
%matplotlib agg

platformIncludeList = []
# Iterate over each geography in data set
for geoid in pep["GEOIDFQ"].unique():

    # Extract the data for a single geography
    temp = pep.loc[pep["GEOIDFQ"] == geoid].copy()

    if(temp.empty):
        continue

    platformIncludeList += [geoid]
    
    # Generate a title string
    geoName = temp.iloc[0]["GEONAME"]
    title = "Composition of Population by Race - {}".format(geoName)
    xlabel = None
    ylabel = "Share of population (percent)"

    # Make the variable names nicer looking
    temp = temp.rename(columns={
        "YEAR":"Year"
    })
    
    # Pivot to wide format
    temp = temp.drop(columns=["GEOIDFQ","GEOTYPE","GEONAME"]).pivot(index="Year", columns="RACE", values="PROPORTION")
    temp.columns.name = None

    # Scale proportions to reflect percentages
    temp = temp*100

    # Create and annotate the plot
    PLOTWIDTH = 8
    fig,ax = plt.subplots(figsize=(PLOTWIDTH,PLOTWIDTH/16*9))

    temp.plot.bar(ax=ax, stacked=True, color=colorset)
   
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_ylim(ymin=0, ymax=100)
    ax.set_yticks([round(tick,0) for tick in ax.get_yticks()])
    handles, labels = ax.get_legend_handles_labels()
    labels = [textwrap.fill(label, 15) for label in labels]
    legend = ax.legend(handles[::-1], labels[::-1], loc='center left', bbox_to_anchor=(1, 0.5), labelspacing=1)
    ax.set_axisbelow(True)
    
    # Format the y-axis labels as integers with comma separators
    ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
   
    # Save the figure to disk as an SVG file
    ax.figure.savefig(os.path.join(chartDir, "{}.svg".format(geoid)), bbox_extra_artists=(legend,), bbox_inches='tight')
    
    plt.close(ax.figure)

    writer = pd.ExcelWriter(os.path.join(chartDir, "{}.xlsx".format(geoid)), engine='xlsxwriter')
    dataOptions = {
        "numberFormat": {
            "Year":"0",
            'White':"0.00", 
            'Asian':"0.00",
            'Black or African American':"0.00",
            'Some other race':"0.00",
            'Two or more races':"0.00",
            'Hispanic or Latino (any races)':"0.00"
        },
        "columnWidth": 40
    }
    chartOptions = {
        "subtype":"stacked",
        "colors": colorset,
        "titles": {
            "chartTitle": title,
            "xTitle": xlabel,
            "yTitle": ylabel
        },
        "seriesOptions": [{"gap":100} for x in temp.columns],
        "xAxisOptions": {
            "num_font": {"size":12},
        },
        "yAxisOptions": {
            "num_font": {"size":12},
            "num_format": "0",
            "name_font": {"size":12},
            "min":0,
            "max":100
        },
        "legendOptions":{
            "position":"bottom",
            "ncol": 2,
            "font":{"size":12}
        },
        "sizeOptions":{
            "x_scale":1.5,
            "y_scale":1.5
        },
        "location": "below"
    }
    morpc.data_chart_to_excel(temp, writer, chartType="column", dataOptions=dataOptions, chartOptions=chartOptions)
    writer.close()    
    
%matplotlib inline

## Generate Insights catalog content

The content in the Insights platform is controlled by a catalog spreadsheet. Each tile to be displayed in the platform must have a record in the catalog.  This section will create the records for the tiles that display the alternative fuel station data.  Eventually this function will be performed by a separate staging script.

First specify the column names used in the catalog.

In [530]:
columnNames=["TileID","TilesetID","GeographyType","GeographyName","Category","Headline","Commentary","ThumbnailURL","Contributor","Vintage","UpdateInterval","ShareURL","DataProductURL","MoreInformationURL"]

Create a new dataframe containing only the geographies for which thumbnail images were produced in the section above.

In [531]:
catalog = pep.loc[pep["GEOIDFQ"].isin(platformIncludeList)].copy()

Extract only the metadata columns of interest and flatten the data to have only one record per geography. Rename the metadata fields to match the catalog fields.

In [532]:
catalog = catalog.filter(items=["GEOIDFQ","GEONAME","GEOTYPE"], axis="columns") \
    .groupby("GEOIDFQ").first() \
    .reset_index() \
    .rename(columns={"GEONAME":"GeographyName","GEOTYPE":"GeographyType"})

Populate some placeholder fields.

In [533]:
catalog["TileID"] = None
catalog["TilesetID"] = "TBD"
catalog["Category"] = None
catalog["Headline"] = "TBD"
catalog["Commentary"] = "TBD"

Generate the URL for the thumbnail images. These will be hosted in GitHub and will be indexed by GEOIDFQ.

In [534]:
catalog["ThumbnailURL"] = catalog["GEOIDFQ"].apply(lambda geoid:"https://raw.githubusercontent.com/morpc-insights/insights-race/refs/heads/main/output_data/charts/{}.svg".format(geoid))

Populate some other simple metadata.  Vintage in this case refers to the year that the content was published in Insights. This is to give readers an idea of how old it is.  UpdateInterval gives them an idea of when to expect the next version. ShareURL is a placeholder for now.

In [535]:
catalog["Contributor"] = "Mid-Ohio Regional Planning Commission"
catalog["Vintage"] = str(datetime.date.today().year)
catalog["UpdateInterval"] = "annually"
catalog["ShareURL"] = None

Generate the data product URL.  This points to an ArcGIS Dashboard that accepts URL parameters.  GEOIDFQ is passed as a parameter to tell the app to load the data for a particular geography.

In [536]:
catalog["DataProductURL"] = catalog["GEOIDFQ"].apply(lambda geoid:"https://www.arcgis.com/apps/dashboards/21d70cd812fb400bb02c82819c593ce5#geoid={}".format(geoid))

Generate the URLs that point to the extended commentary pages.  Default to a common page (population.pdf) hosted in GitHub.  Point to specific pages for the 15-county region and for each county.

In [537]:
catalog["MoreInformationURL"] = "https://morpc1-my.sharepoint.com/:w:/g/personal/aporr_morpc_org/EU33sTbsyapHgRO8yWTMz_cB-jkrxryC7h6PZkEoPwY3xQ?e=2E3FR7"

Extract only the required columns.

In [538]:
catalog = catalog.filter(items=columnNames, axis="columns")

Inspect the listing.

In [539]:
catalog.head()

,TileID,TilesetID,GeographyType,GeographyName,Category,Headline,Commentary,ThumbnailURL,Contributor,Vintage,UpdateInterval,ShareURL,DataProductURL,MoreInformationURL
0,None,TBD,County,Delaware County,None,TBD,TBD,https://raw.githubusercontent.com/morpc-insigh...,Mid-Ohio Regional Planning Commission,2025,annually,None,https://www.arcgis.com/apps/dashboards/21d70cd...,https://morpc1-my.sharepoint.com/:w:/g/persona...
1,None,TBD,County,Fairfield County,None,TBD,TBD,https://raw.githubusercontent.com/morpc-insigh...,Mid-Ohio Regional Planning Commission,2025,annually,None,https://www.arcgis.com/apps/dashboards/21d70cd...,https://morpc1-my.sharepoint.com/:w:/g/persona...
2,None,TBD,County,Fayette County,None,TBD,TBD,https://raw.githubusercontent.com/morpc-insigh...,Mid-Ohio Regional Planning Commission,2025,annually,None,https://www.arcgis.com/apps/dashboards/21d70cd...,https://morpc1-my.sharepoint.com/:w:/g/persona...
3,None,TBD,County,Franklin County,None,TBD,TBD,https://raw.githubusercontent.com/morpc-insigh...,Mid-Ohio Regional Planning Commission,2025,annually,None,https://www.arcgis.com/apps/dashboards/21d70cd...,https://morpc1-my.sharepoint.com/:w:/g/persona...
4,None,TBD,County,Hocking County,None,TBD,TBD,https://raw.githubusercontent.com/morpc-insigh...,Mid-Ohio Regional Planning Commission,2025,annually,None,https://www.arcgis.com/apps/dashboards/21d70cd...,https://morpc1-my.sharepoint.com/:w:/g/persona...


Save the catalog to an Excel spreadsheet.

In [540]:
catalog.to_excel("catalog.xlsx", index=False)

It is necessary to manually add these records to the master catalog or update the records already therein.  See the following file in GitHub:

https://github.com/morpc/morpc-insights/blob/main/catalog/morpc_insights_catalog.xlsx


In [541]:
catalog

,TileID,TilesetID,GeographyType,GeographyName,Category,Headline,Commentary,ThumbnailURL,Contributor,Vintage,UpdateInterval,ShareURL,DataProductURL,MoreInformationURL
0,None,TBD,County,Delaware County,None,TBD,TBD,https://raw.githubusercontent.com/morpc-insigh...,Mid-Ohio Regional Planning Commission,2025,annually,None,https://www.arcgis.com/apps/dashboards/21d70cd...,https://morpc1-my.sharepoint.com/:w:/g/persona...
1,None,TBD,County,Fairfield County,None,TBD,TBD,https://raw.githubusercontent.com/morpc-insigh...,Mid-Ohio Regional Planning Commission,2025,annually,None,https://www.arcgis.com/apps/dashboards/21d70cd...,https://morpc1-my.sharepoint.com/:w:/g/persona...
2,None,TBD,County,Fayette County,None,TBD,TBD,https://raw.githubusercontent.com/morpc-insigh...,Mid-Ohio Regional Planning Commission,2025,annually,None,https://www.arcgis.com/apps/dashboards/21d70cd...,https://morpc1-my.sharepoint.com/:w:/g/persona...
3,None,TBD,County,Franklin County,None,TBD,TBD,https://raw.githubusercontent.com/morpc-insigh...,Mid-Ohio Regional Planning Commission,2025,annually,None,https://www.arcgis.com/apps/dashboards/21d70cd...,https://morpc1-my.sharepoint.com/:w:/g/persona...
4,None,TBD,County,Hocking County,None,TBD,TBD,https://raw.githubusercontent.com/morpc-insigh...,Mid-Ohio Regional Planning Commission,2025,annually,None,https://www.arcgis.com/apps/dashboards/21d70cd...,https://morpc1-my.sharepoint.com/:w:/g/persona...
5,None,TBD,County,Knox County,None,TBD,TBD,https://raw.githubusercontent.com/morpc-insigh...,Mid-Ohio Regional Planning Commission,2025,annually,None,https://www.arcgis.com/apps/dashboards/21d70cd...,https://morpc1-my.sharepoint.com/:w:/g/persona...
6,None,TBD,County,Licking County,None,TBD,TBD,https://raw.githubusercontent.com/morpc-insigh...,Mid-Ohio Regional Planning Commission,2025,annually,None,https://www.arcgis.com/apps/dashboards/21d70cd...,https://morpc1-my.sharepoint.com/:w:/g/persona...
7,None,TBD,County,Logan County,None,TBD,TBD,https://raw.githubusercontent.com/morpc-insigh...,Mid-Ohio Regional Planning Commission,2025,annually,None,https://www.arcgis.com/apps/dashboards/21d70cd...,https://morpc1-my.sharepoint.com/:w:/g/persona...
8,None,TBD,County,Madison County,None,TBD,TBD,https://raw.githubusercontent.com/morpc-insigh...,Mid-Ohio Regional Planning Commission,2025,annually,None,https://www.arcgis.com/apps/dashboards/21d70cd...,https://morpc1-my.sharepoint.com/:w:/g/persona...
9,None,TBD,County,Marion County,None,TBD,TBD,https://raw.githubusercontent.com/morpc-insigh...,Mid-Ohio Regional Planning Commission,2025,annually,None,https://www.arcgis.com/apps/dashboards/21d70cd...,https://morpc1-my.sharepoint.com/:w:/g/persona...
